In [1]:
# %pip install requirements-parser
# %pip install Levenshtein

In [2]:
import os
os.getcwd()
os.chdir('/home/gjungwirth/data/02_data/')

In [3]:
import requirements
args = {}
args['input'] = '/home/gjungwirth/data/02_data/03_analyses/library_versions/requirements-files'
args['storage_file'] = '/home/gjungwirth/data/02_data/03_analyses/library_versions/python_data.bak'
req_files = []
with open(args['input']) as f:
    req_files = [line.rstrip() for line in f]
req_files[0:1]

['repos/blackrobot_dotfiles/lib/python/requirements.txt']

In [4]:
all_versions = {}
for reqfile in req_files: #[0:20]:
    try:
        with open(reqfile) as f:
            for req in requirements.parse(f):
                #print(req)
                #print(req.name, req.specs, req.extras)
                if req.name not in all_versions:
                    all_versions[req.name] = [req.specs]
                else:
                    all_versions[req.name].append(req.specs)
    except:
        pass

/home/gjungwirth/data/02_data/.venv/lib/python3.9/site-packages/requirements/parser.py:44: UserWarning: Private repos not supported. Skipping.
  warnings.warn('Private repos not supported. Skipping.')


In [5]:
print("Total unique deps:", len(all_versions.keys()))

sorted_versions = sorted(all_versions, key=lambda k: len(all_versions[k]), reverse=True)
for top in sorted_versions[0:50]:
    v = [v for v in all_versions[top] if len(v) > 0]
    print(top, "({})".format(len(all_versions[top])), v[0:5])

Total unique deps: 3028
flake8 (255) [[('>=', '3.0')], [('==', '3.7.6')], [('==', '2.5.1')], [('==', '3.7.9')], [('==', '3.7.8')]]
requests (248) [[('>=', '2.9')], [('>=', '2.9')], [('==', '2.21.0')], [('==', '2.24.0')], [('==', '2.12.4')]]
six (185) [[('==', '1.10.0')], [('==', '1.15.0')], [('==', '1.13.0')], [('==', '1.12.0')], [('==', '1.10.0')]]
ipython (165) [[('<', '6')], [('==', '5.1.0')], [('==', '4.0.1')], [('==', '7.7.0')], [('==', '7.8.0')]]
virtualenv (150) [[('==', '1.11.4')], [('==', '20.0.26')], [('==', '15.1.0')], [('==', '1.11.6')], [('==', '16.1.0')]]
jedi (150) [[('==', '0.15.2')], [('==', '0.14.1')], [('==', '0.8.1-final0')], [('==', '0.13.1')], [('==', '0.9.0')]]
numpy (140) [[('==', '1.18.5')], [('==', '1.17.1')], [('==', '1.8.2')], [('==', '1.15.4')], [('==', '1.12.1')]]
pylint (139) [[('==', '2.4.4')], [('==', '2.6.0')], [('==', '2.5.3')], [('==', '1.9.1')], [('==', '1.5.4')]]
pytest (131) [[('==', '3.2.3')], [('==', '5.3.2')], [('==', '2.3.4')], [('==', '2.3.4'

## Analysis with pd DataFrame



In [6]:
import pandas as pd

#pd.DataFrame.from_dict(all_versions, orient='index',columns=['foo','bar'])
df = pd.DataFrame(all_versions.items(), columns=['unique_libs','version_specifiers'])

df

,unique_libs,version_specifiers
0,click,"[[], [], [(==, 7.0)], [(==, 6.7)], [(==, 7.1.2..."
1,awscli,"[[], [], [(>=, 1.7)], [], [(==, 1.18.126)], [(..."
2,ipython,"[[], [], [], [], [(<, 6)], [], [], [(==, 5.1.0..."
3,json2yaml,[[]]
4,postmarker,[[]]
...,...,...
3023,usb,[[]]
3024,google-crc32c,"[[(==, 1.0.0)]]"
3025,korean-lunar-calendar,"[[(==, 0.2.1)]]"
3026,proto-plus,"[[(==, 1.13.0)]]"


In [7]:
import requests

# contains: either error OR: author, versions, keywords, description, name
def get_lib_info_pypi(libname):
    res = requests.get('https://pypi.org/pypi/{}/json'.format(libname))
    if res.status_code < 300:
        res_json = res.json()
        del res_json['urls']
        res_json['releases'] = res_json['releases'].keys() # reduce amount of information a bit
        return res_json
    else:
        return {'error': f'Status code: {res.status_code}'}

#libinfo = get_lib_info_pypi('not_found_saldkfja')

In [8]:
#df['libinfo'] = df['unique_libs'].apply(get_lib_info_pypi)

def alter_libinfo(item):
    if 'releases' in item:
        item['releases'] = list(item['releases'])
    return item
#df['libinfo'].apply(alter_libinfo)
#df.to_pickle(args['storage_file'])

df2 = pd.read_pickle(args['storage_file'])
df = df2
#df2.equals(df)

In [9]:
df['private_lib'] = df['libinfo'].apply(lambda a: 'error' in a.keys())

tmp = df[df['private_lib'] == True]

#print("\n".join(tmp['unique_libs']))

## work on distances

In [10]:
from Levenshtein import distance
from itertools import product
import numpy as np

lib_names = list(df[df.unique_libs.notnull()].unique_libs.values)
dist = [distance(*x) for x in product(lib_names, repeat=2)]

In [11]:
dist_df = pd.DataFrame(np.array(dist).reshape(len(lib_names), len(lib_names)), index=lib_names, columns=lib_names)
dist_df['closest_neighbor'] = dist_df.applymap(lambda x: None if x==0 else x).idxmin(axis="columns")

In [12]:
dist_df['dist_of_neighbor'] = dist_df.apply(lambda x: x[x['closest_neighbor']],axis=1)
dist_df['dist_of_neighbor']

click                     1
awscli                    3
ipython                   1
json2yaml                 4
postmarker                4
                         ..
usb                       2
google-crc32c             6
korean-lunar-calendar    13
proto-plus                4
PyMeeus                   4
Name: dist_of_neighbor, Length: 3027, dtype: int64

In [13]:
libs_w_distances = df.set_index('unique_libs').join(dist_df[['closest_neighbor','dist_of_neighbor']])

In [14]:
tmp = libs_w_distances[libs_w_distances['private_lib'] == True].sort_values(by='dist_of_neighbor')

## Links

https://stackoverflow.com/questions/29919306/find-the-column-name-which-has-the-maximum-value-for-each-row
https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#

https://stackoverflow.com/questions/49737760/is-there-a-slower-or-more-controlled-alternative-to-apply

https://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-dataframe-in-pandas


https://stackoverflow.com/questions/38309729/count-unique-values-with-pandas-per-groups

https://pandas.pydata.org/docs/user_guide/merging.html  
note: all documentation on this page (official) is very good

https://stackoverflow.com/questions/47152344/how-to-calculate-levenshtein-ratio-distance-for-rows-in-my-column-in-python

